Давайте проанализируем данные опроса 4361 женщин из Ботсваны:

botswana.tsv
О каждой из них мы знаем:

сколько детей она родила (признак ceb)
возраст (age)
длительность получения образования (educ)
религиозная принадлежность (religion)
идеальное, по её мнению, количество детей в семье (idlnchld)
была ли она когда-нибудь замужем (evermarr)
возраст первого замужества (agefm)
длительность получения образования мужем (heduc)
знает ли она о методах контрацепции (knowmeth)
использует ли она методы контрацепции (usemeth)
живёт ли она в городе (urban)
есть ли у неё электричество, радио, телевизор и велосипед (electric, radio, tv, bicycle)
Давайте научимся оценивать количество детей ceb по остальным признакам.

Загрузите данные и внимательно изучите их. Сколько разных значений принимает признак religion?


In [88]:
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt

In [89]:
data=pd.read_csv('Week3/botswana.tsv','\t')
data.religion.describe()

count       4361
unique         4
top       spirit
freq        1841
Name: religion, dtype: object

In [90]:
print "Answer 2 %i" %(1834)

Answer 2 1834


Во многих признаках есть пропущенные значения. Сколько объектов из 4361 останется, если выбросить все, содержащие пропуски?

In [91]:
data1 = data.dropna()
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1834 entries, 1 to 4360
Data columns (total 15 columns):
ceb         1834 non-null int64
age         1834 non-null int64
educ        1834 non-null int64
religion    1834 non-null object
idlnchld    1834 non-null float64
knowmeth    1834 non-null float64
usemeth     1834 non-null float64
evermarr    1834 non-null int64
agefm       1834 non-null float64
heduc       1834 non-null float64
urban       1834 non-null int64
electric    1834 non-null float64
radio       1834 non-null float64
tv          1834 non-null float64
bicycle     1834 non-null float64
dtypes: float64(9), int64(5), object(1)
memory usage: 229.2+ KB


В разных признаках пропуски возникают по разным причинам и должны обрабатываться по-разному.

Например, в признаке agefm пропуски стоят только там, где evermarr=0, то есть, они соответствуют женщинам, никогда не выходившим замуж. Таким образом, для этого признака NaN соответствует значению "не применимо".

В подобных случаях, когда признак x1 на части объектов в принципе не может принимать никакие значения, рекомендуется поступать так:

создать новый бинарный признак
x2={1,0,x1='не применимо',иначе;
заменить "не применимо" в x1 на произвольную константу c, которая среди других значений x1 не встречается.
Теперь, когда мы построим регрессию на оба признака и получим модель вида
y=β0+β1x1+β2x2,
на тех объектах, где x1 было измерено, регрессионное уравнение примет вид
y=β0+β1x,
а там, где x1 было "не применимо", получится
y=β0+β1c+β2.
Выбор c влияет только на значение и интерпретацию β2, но не β1.

Давайте используем этот метод для обработки пропусков в agefm и heduc.

Создайте признак nevermarr, равный единице там, где в agefm пропуски.
Удалите признак evermarr — в сумме с nevermarr он даёт константу, значит, в нашей матрице X будет мультиколлинеарность.
Замените NaN в признаке agefm на cagefm=0.
У объектов, где nevermarr = 1, замените NaN в признаке heduc на cheduc1=−1 (ноль использовать нельзя, так как он уже встречается у некоторых объектов выборки).
Сколько осталось пропущенных значений в признаке heduc?

In [92]:
data['newermarr'] = data['agefm'].apply(lambda x : 1 if pd.isnull(x) else 0)
data.drop('evermarr', axis=1, inplace=True)
data['agefm'].fillna(0, inplace=True)

In [93]:
data['heduc'][data['newermarr']==1] = data['heduc'][data['newermarr']==1].apply(
    lambda x: -1 if pd.isnull(x) else x)

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [94]:
data[pd.isnull(data['heduc']) ].count()

ceb          123
age          123
educ         123
religion     123
idlnchld     116
knowmeth     122
usemeth      121
agefm        123
heduc          0
urban        123
electric     123
radio        123
tv           123
bicycle      123
newermarr    123
dtype: int64

In [96]:
print 'Answer 3 - %i' % (123)
print data.shape

Answer 3 - 123
(4361, 15)


Избавимся от оставшихся пропусков.

Для признаков idlnchld, heduc и usemeth проведите операцию, аналогичную предыдущей: создайте индикаторы пропусков по этим признакам (idlnchld_noans, heduc_noans, usemeth_noans), замените пропуски на нехарактерные значения (cidlnchld=−1, cheduc2=−2 (значение -1 мы уже использовали), cusemeth=−1).

Остались только пропуски в признаках knowmeth, electric, radio, tv и bicycle. Их очень мало, так что удалите объекты, на которых их значения пропущены.

Какого размера теперь наша матрица данных? Умножьте количество строк на количество всех столбцов (включая отклик ceb).

In [97]:
data['idlnchld_noans'] = data['idlnchld'].apply(lambda x : 1 if pd.isnull(x) else 0)
data['heduc_noans']= data['heduc'].apply(lambda x : 1 if pd.isnull(x) else 0)
data['usemeth_noans'] = data['usemeth'].apply(lambda x : 1 if pd.isnull(x) else 0)
data.idlnchld.fillna(-1,inplace=True)
data.heduc.fillna(-2,inplace=True)
data.usemeth.fillna(-1,inplace=True)
data[pd.isnull(data['usemeth'])]

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,newermarr,idlnchld_noans,heduc_noans,usemeth_noans


In [98]:
print data.shape[0]*data.shape[1]
print data.shape

78498
(4361, 18)


In [99]:
data=data.dropna()

In [101]:
print 'Answer 4 - %i' %(data.shape[0]*data.shape[1])
print data.shape

Answer 4 - 78264
(4348, 18)


Постройте регрессию количества детей ceb на все имеющиеся признаки методом smf.ols, как в разобранном до этого примере. Какой получился коэффициент детерминации R2? Округлите до трёх знаков после десятичной точки.

In [104]:
print data.columns.values

['ceb' 'age' 'educ' 'religion' 'idlnchld' 'knowmeth' 'usemeth' 'agefm'
 'heduc' 'urban' 'electric' 'radio' 'tv' 'bicycle' 'newermarr'
 'idlnchld_noans' 'heduc_noans' 'usemeth_noans']


In [105]:
m1 = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm + '\
    ' heduc + urban + electric + radio + tv + bicycle + newermarr + idlnchld_noans + '\
    ' heduc_noans + usemeth_noans',
             data=data)
fitted = m1.fit()
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Mon, 08 May 2017   Prob (F-statistic):               0.00
Time:                        17:45:45   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------
Intercept                 -1

In [106]:
print 'Answer 5 - %f' %(0.644)

Answer 5 - 0.644000


Обратите внимание, что для признака religion в модели автоматически создалось несколько бинарных фиктивных переменных. Сколько их?

In [107]:
print 'Answer 6 - %i' % (3)

Answer 6 - 3


Проверьте критерием Бройша-Пагана гомоскедастичность ошибки в построенной модели. Выполняется ли она?

Если ошибка гетероскедастична, перенастройте модель, сделав поправку Уайта типа HC1.

In [108]:
print 'Breusch-Pagan test: p=%f' % sms.het_breushpagan(fitted.resid, fitted.model.exog)[1]

Breusch-Pagan test: p=0.000000


In [109]:
print 'Answer 7 - geteroscedast'

Answer 7 - geteroscedast


In [110]:
m2 = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm + '\
    ' heduc + urban + electric + radio + tv + bicycle + newermarr + idlnchld_noans + '\
    ' heduc_noans + usemeth_noans',
             data=data)
fitted = m2.fit(cov_type='HC1')
print fitted.summary()
print 'Breusch-Pagan test: p=%f' % sms.het_breushpagan(fitted.resid, fitted.model.exog)[1]

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     345.0
Date:                Tue, 09 May 2017   Prob (F-statistic):               0.00
Time:                        00:10:39   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:                  HC1                                         
                             coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------
Intercept                 -1

Удалите из модели незначимые признаки religion, radio и tv. Проверьте гомоскедастичность ошибки, при необходимости сделайте поправку Уайта.

Не произошло ли значимого ухудшения модели после удаления этой группы признаков? Проверьте с помощью критерия Фишера. Чему равен его достигаемый уровень значимости? Округлите до четырёх цифр после десятичной точки.

Если достигаемый уровень значимости получился маленький, верните все удалённые признаки; если он достаточно велик, оставьте модель без религии, тв и радио.

In [113]:
m3 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + '\
    ' heduc + urban + electric + bicycle + newermarr + idlnchld_noans + '\
    ' heduc_noans + usemeth_noans',
             data=data)
fitted = m3.fit(cov_type='HC1')
print fitted.summary()
print 'Breusch-Pagan test: p=%f' % sms.het_breushpagan(fitted.resid, fitted.model.exog)[1]
print "F=%f, p=%f, k1=%f" % m2.fit().compare_f_test(m3.fit())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Tue, 09 May 2017   Prob (F-statistic):               0.00
Time:                        00:16:11   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

Признак usemeth_noans значим по критерию Стьюдента, то есть, при его удалении модель значимо ухудшится. Но вообще-то отдельно его удалять нельзя: из-за того, что мы перекодировали пропуски в usemeth произвольно выбранным значением cusemeth=−1, удалять usemeth_noans и usemeth можно только вместе.

Удалите из текущей модели usemeth_noans и usemeth. Проверьте критерием Фишера гипотезу о том, что качество модели не ухудшилось. Введите номер первой значащей цифры в достигаемом уровне значимости (например, если вы получили 5.5×10−8, нужно ввести 8).

Если достигаемый уровень значимости получился маленький, верните удалённые признаки; если он достаточно велик, оставьте модель без usemeth и usemeth_noans.

In [118]:
m4 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + '\
    ' heduc + urban + electric + bicycle + newermarr + idlnchld_noans + '\
    ' heduc_noans + usemeth_noans',
             data=data)
fitted = m4.fit()
print fitted.summary()
print 'Breusch-Pagan test: p=%f' % sms.het_breushpagan(fitted.resid, fitted.model.exog)[1]
print "F=%f, p=%e, k1=%f" % m3.fit().compare_f_test(m4.fit())
fitted = m4.fit(cov_type='HC1')
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     559.5
Date:                Tue, 09 May 2017   Prob (F-statistic):               0.00
Time:                        00:31:37   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.198     -5.

Посмотрите на доверительные интервалы для коэффициентов итоговой модели (не забудьте использовать поправку Уайта, если есть гетероскедастичность ошибки) и выберите правильные выводы.

С увеличением возраста женщины на 1 год среднее количество детей возрастает на 0.17 (p<0.001, 95% доверительный интервал — [0.16, 0.18])

У женщин, не знающих, какое количество детей идеально, в среднем на
βidlnchld_noans+cidlnchldβidlnchld≈0.58
детей больше

У женщин, знакомых с методами контрацепции, при прочих равных в среднем на 0.6 ребёнка меньше (p=0.001, 95% доверительный интервал для разницы между средними — [-0.9, -0.2])

У женщин, не знающих, какое количество детей идеально, в среднем на 0.66 ребёнка больше (p=0.002, 95% доверительный интервал — [0.2, 1.1])

Итоговая модель объясняет 63% вариации отклика

У женщин, никогда не выходивших замуж, при прочих равных в среднем на 2.3 ребёнка меньше (p<0.001, 95% доверительный интервал для разницы между средними — [-2.6, -1.9])